# Commonly Used Loss Functions

The model paramters of Neural Networks are trained using a process known as the back-propagation of errors from a loss function. So model training requires that you choose a loss function when designing your model. 

In this chapter, we'll walk through a few commonly used loss function available in PyTorch and what's involved in the computation. 

At a bisic level, Loss Functions can be categorized into two categories: Classification Loss and Regression Loss. 

Below we show examples of both.


In [1]:
import torch
import torch.nn as nn

print(f'Torch version: {torch.__version__}')

Torch version: 1.7.1


## Regression Analysis with a Regression Loss Function
Regression analysis predicts a continuous output variable based on the value of one or more input variables. Briefly, the goal of regression model is to build a mathematical approximation that defines the output variable as a function of the input variables. An example of a continuous variable is: the predicted price of a home based on inputs such as living space, number of rooms, etc. Note, the output is a continuous value, not a discrete value or a categorical value. The input variable can be continuous ro discrete.

The loss fucntion associated with regression analysis is known as the regression loss. Mean Square Error (MSE) is the most commonly used regression loss function (as we'll illustrate that here with an example). MSE is the sum of squared distances between the target variable and predicted values.

MSE is calculated as the mean of squared difference between predictions and actual observations (also know as the target). Expressed as a computaiton, it is:

`sum( square( predicted_vals - targets ) ) / (number of training samples)` and you'll see it expressed as PyTorch code below.

Intuitively, the squaring of the error term means that larger deviations from the target values amplify the error way more than than smaller deviations, meaning that the model is penalized more for deviating more from the target values.


In [2]:
predicted_vals= torch.randn(3, 5, requires_grad=True)
targets = torch.randn(3, 5)
regression_mse_loss = nn.MSELoss()(predicted_vals, targets)  # calculates mean
print('predicted_vals:\n\t', predicted_vals)
print('targets:\n\t', targets)
print('regression loss:\n\t', regression_mse_loss)
hand_calc_regress_mse_loss = torch.sum(torch.square(predicted_vals - targets)) / (targets.shape[0] * targets.shape[1])
print('hand calculated regression loss:\n\t:', hand_calc_regress_mse_loss)
assert torch.equal(regression_mse_loss, hand_calc_regress_mse_loss)

predicted_vals:
	 tensor([[ 1.0039, -0.6457, -1.3066, -0.6071, -0.1067],
        [-0.1304,  2.0721,  0.2749,  0.1164, -0.8147],
        [-0.5657,  0.9203, -0.3077,  1.4288, -3.7896]], requires_grad=True)
targets:
	 tensor([[ 0.6040, -0.7033,  1.2591,  0.6451,  0.6520],
        [-0.1254,  0.9277, -1.2089, -0.0797, -0.5148],
        [-0.1149, -0.0926,  0.1163,  0.2851,  0.8966]])
regression loss:
	 tensor(2.4805, grad_fn=<MseLossBackward>)
hand calculated regression loss:
	: tensor(2.4805, grad_fn=<DivBackward0>)


## Binary Classification with a Cross Entropy Loss Function

Measuring the Binary Cross Entropy Loss between the target and the predicted output is done with: 

`loss = -1 * (target * log(predicted)  +  (1 - target) * log(1 - predicted))` where log means log-base-e

We show below with a simple example that the Cross Entropy Loss defined above penalizes misclassifications more, a property that can help in better learning.

We compute the MSE Loss and the Cross Entropy Loss for a correctly predicted value (0.95) and a wrongly predicted value (0.1), i.e., a misclassification. In both cases the target value is 1.

Below we calculate the MSE loss.

| Target &nbsp &nbsp     | Predicted &nbsp &nbsp| MSE Loss | &nbsp &nbsp Comment | 
| :---:        |    :----:   |          :---: | :--- |
| 1      | 0.95       | (1.0 - 0.95)**2 = 0.0025   | &nbsp &nbsp For a correct predictin, loss is small |
| 1   | 0.1        | (1.0 - 0.1)**2 = 0.81      | &nbsp &nbsp For a wrong predictiob, loss is big

For the Cross Entropy Loss, the computation reduces to `-1 * target * log(predicted)` or simply `-1 * log(predicted)` since target value is 1.

| Target &nbsp &nbsp | Predicted &nbsp &nbsp| Cross Entropy Loss| &nbsp &nbsp Comment | 
| :---:        |    :----:   |          :---: | :--- |
| 1      | 0.95       | -1 * log(0.95) =  0.051  | &nbsp &nbsp Prediction is correct; loss is small |
| 1   | 0.1        | -1 * log(0.1) = 2.302      | &nbsp &nbsp Prediction is wrong; loss is LARGE

If the predicted output is close to the desired output, then the loss is small (for both loss functions). The difference is noticeable however when the output is misclassified (0.81 for MSE Loss and 2.302 for Cross Entropy Loss). 

The Cross Entropy Loss function also has the benefit of learning at a faster pace. To learn (update model parameters), we back-propagate the loss, and that is done by taking the partial derivative of the loss with respect to the weights. In doing so, we can show that the rate at which the model parameters (weights and biases) learn is proportional to `output - target`, i.e., proportional to the error in the output. The larger the error, the faster the model will learn which is a very nice property.

In the example below, we create two clusters of normalized data points 1.5 units apart. Each cluster belongs a class; so we have two classes, class 0 and class 1. The loss computation uses `nn.BCELoss`, the PyTorch Binary Cross Entropy Loss function.

It is typical to use the nn.Sigmoid activation as the output layer followed by the nn.BCELoss to calculate the Binary Cross Entropy loss.

We also calculate is by hand and ensure we get the same results.



In [3]:
input = torch.cat([torch.randn(3), torch.randn(3) + 1.5])  # two clusters 1.5 units apart
target = torch.cat([torch.zeros(3), torch.ones(3)])  # ... from two classes; class 0, class 1
predicted = torch.sigmoid(input)

unreduced_loss = nn.BCELoss(reduction='none')(predicted, target)
mean_loss = nn.BCELoss(reduction='mean')(predicted, target)
print('input:\n\t', input)
print('target:\n\t', target)
print('predicted:\n\t', predicted)
print('unreduced loss:\n\t', unreduced_loss)
print('mean loss:\n\t', mean_loss)

hand_calc_loss = -1 * (target * torch.log(predicted) + (1 - target) * torch.log(1 - predicted))
print('hand calculated loss\n\t', hand_calc_loss)
assert torch.equal(hand_calc_loss, unreduced_loss)  # check they are the same

input:
	 tensor([-0.7816,  0.8957,  0.4969,  2.9781,  3.1644,  2.4352])
target:
	 tensor([0., 0., 0., 1., 1., 1.])
predicted:
	 tensor([0.3140, 0.7101, 0.6217, 0.9516, 0.9595, 0.9195])
unreduced loss:
	 tensor([0.3768, 1.2381, 0.9722, 0.0496, 0.0414, 0.0840])
mean loss:
	 tensor(0.4603)
hand calculated loss
	 tensor([0.3768, 1.2381, 0.9722, 0.0496, 0.0414, 0.0840])


In the exampe above, the Cross Entropy Loss calculation was done in two steps.
1. A Sigmoid lon-linearity on the output
2. Application of the Binary Corss WEntropy Loss

For numerical stability reasons, the two steps above can be collapsed into one step with the `nn.BCEWithLogitsLoss` loss function.
`nn.BCEWithLogitsLoss` loss combines a Sigmoid layer and the BCELoss in one single class

In the example below, we reuse the input tensor from above and show the equivalence.


In [5]:
# inputs and target same as above
loss_unreduced = nn.BCEWithLogitsLoss(reduction='none')(input, target)
loss_mean = nn.BCEWithLogitsLoss(reduction='mean')(input, target)
print('unreduced loss:\n', unreduced_loss)
print('mean loss:\n', mean_loss)  # will be the same as the two step calculation

unreduced loss:
 tensor([0.8561, 0.6236, 1.5947, 0.1294, 0.5867, 0.3515])
mean loss:
 tensor(0.6903)


## Multi-Class Classification with a Cross Entropy Loss Function

We use this loss function when we train a model to output a probability over multiple classes(multi-class classification). 

To be concrete, let's ay we're classifying images of digits. It is used for .

It is useful when training a classification problem with C classes.
The `target` that this loss expects should be a class index in the range :math:`[0, C-1]` where `C = number of classes`; 

References
https://forums.fast.ai/t/nllloss-implementation/20028/

https://gombru.github.io/2018/05/23/cross_entropy_loss/

https://www.youtube.com/watch?v=7q7E91pHoW4&ab_channel=PythonEngineer




In [4]:

input = torch.randn(7, 5)
target = torch.randint(5, (7,))
# predicted = torch.softmax(input, dim=-1)
predicted = nn.LogSoftmax()(input)
unreduced_loss = nn.NLLLoss(reduction='none')(predicted, target)
mean_loss = nn.NLLLoss(reduction='mean')(predicted, target)
print('input:\n\t', input)
print('target:\n\t', target)
print('predicted:\n\t', predicted)
print('unreduced loss:\n\t', unreduced_loss)
print('mean loss:\n\t', mean_loss)

def coded_nll_loss(logs, targets):
    out = torch.zeros_like(targets, dtype=torch.float)
    for i in range(len(targets)):
        out[i] = logs[i][targets[i]]
    # out = torch.diag(logs[:,targets])  # one-liner instead of loop
    return -out  # negative out
hand_coded = coded_nll_loss(predicted, target)
print('hand calc loss', hand_coded) 

input:
	 tensor([[-1.0378e+00,  3.8241e-01,  7.4357e-01, -1.5864e-01,  1.2322e+00],
        [-3.0680e-01,  4.0609e-01, -4.5084e-01, -7.6833e-01,  3.6217e-02],
        [-1.5142e+00,  1.0059e-01,  2.3358e-02,  1.1518e+00, -4.5974e-01],
        [-1.4887e+00, -5.6808e-01, -1.8939e+00,  1.7082e-01,  4.6464e-01],
        [-5.4393e-01,  1.1777e+00,  1.6071e+00,  2.7852e-02, -8.0960e-01],
        [ 1.6506e+00, -1.2790e+00,  2.6135e-01,  6.1170e-01, -7.3882e-04],
        [ 2.8466e+00, -8.0100e-01,  3.4753e-01, -3.0685e-01,  7.6342e-01]])
target:
	 tensor([1, 2, 1, 2, 2, 3, 3])
predicted:
	 tensor([[-3.1427, -1.7224, -1.3613, -2.2635, -0.8726],
        [-1.7826, -1.0697, -1.9266, -2.2441, -1.4396],
        [-3.3298, -1.7150, -1.7923, -0.6638, -2.2754],
        [-2.8025, -1.8819, -3.2077, -1.1430, -0.8492],
        [-2.8750, -1.1534, -0.7240, -2.3032, -3.1407],
        [-0.6143, -3.5439, -2.0035, -1.6532, -2.2656],
        [-0.2433, -3.8909, -2.7424, -3.3968, -2.3265]])
unreduced loss:
	 tensor([

In [5]:
# input and target, same as above
loss = nn.CrossEntropyLoss()(input, target)
print('unreduced loss:\n', unreduced_loss)
print('mean loss:\n', mean_loss)

unreduced loss:
 tensor([1.7224, 1.9266, 1.7150, 3.2077, 0.7240, 1.6532, 3.3968])
mean loss:
 tensor(2.0494)


## Multi-Label Categorical Classification


In [6]:
input = torch.randn(7, 5)
target = torch.randint(2, (7, 5), dtype=torch.float)
predicted = torch.sigmoid(input)
unreduced_loss = nn.BCELoss(reduction='none')(predicted, target)
mean_loss = nn.BCELoss(reduction='mean')(predicted, target)
print('input:\n', input)
print('target:\n', target)
print('predicted:\n', predicted)
print('unreduced loss:\n', unreduced_loss)
print('mean loss:\n', mean_loss)

input:
 tensor([[-1.2502, -1.1392,  0.4723,  1.1843,  2.1823],
        [-1.4041,  0.2953, -0.7898,  1.0377, -0.0668],
        [-0.4266,  0.6901,  1.1937,  0.2577, -0.8288],
        [-0.9240,  1.6145,  0.7464,  0.7444, -1.3918],
        [ 1.5455,  1.4927,  0.3397,  2.5424, -0.6971],
        [ 0.7288, -0.5748,  1.5685, -0.3264,  0.6863],
        [ 0.1455, -0.5103, -0.2337, -0.7066, -0.1487]])
target:
 tensor([[0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 1.],
        [0., 0., 0., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0.],
        [0., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1.]])
predicted:
 tensor([[0.2227, 0.2425, 0.6159, 0.7657, 0.8986],
        [0.1972, 0.5733, 0.3122, 0.7384, 0.4833],
        [0.3949, 0.6660, 0.7674, 0.5641, 0.3039],
        [0.2841, 0.8340, 0.6784, 0.6780, 0.1991],
        [0.8243, 0.8165, 0.5841, 0.9271, 0.3325],
        [0.6745, 0.3601, 0.8276, 0.4191, 0.6652],
        [0.5363, 0.3751, 0.4418, 0.3304, 0.4629]])
unreduced loss:
 tensor

In [7]:
# inputs and targets ramins the same as above
unreduced_loss = nn.BCEWithLogitsLoss(reduction='none')(input, target)
mean_loss = nn.BCELoss(reduction='mean')(predicted, target)
print('unreduced loss:\n', unreduced_loss)
print('mean loss:\n', mean_loss)

unreduced loss:
 tensor([[0.2519, 0.2777, 0.9569, 0.2669, 0.1069],
        [1.6237, 0.5563, 0.3743, 1.3410, 0.7271],
        [0.5024, 1.0966, 1.4585, 0.5726, 1.1911],
        [1.2583, 0.1815, 0.3880, 0.3887, 1.6138],
        [0.1933, 0.2028, 0.8773, 2.6182, 0.4042],
        [1.1225, 1.0213, 0.1893, 0.8696, 0.4077],
        [0.6230, 0.9805, 0.5831, 1.1076, 0.7702]])
mean loss:
 tensor(0.7744)


In [8]:
assert torch.equal(torch.tensor([1, 2]), torch.tensor([1, 2]))